### Etapas iniciais: importação de pacotes, definição de funções, carregamento de dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Carregamento dos datasets
winemag = pd.read_csv('datasets/winemag_red.csv')
vivino = pd.read_csv('datasets/vivino_red.csv')

In [3]:
def values_missing(df, only_missing=True):
    '''Calcula e retorna o percentual e o número de registros nulos no dataframe df'''
    missing = pd.concat({'percent_missing': 100* df.isnull().sum() / len(df),
                         'rows_missing': df.isnull().sum()}, axis=1)
    if only_missing:
        missing = missing[missing.percent_missing > 0]
    return missing.sort_values(by='percent_missing', ascending=False)


## Junção dos datasets da Wine Enthusiast e do Vivino
- A junção dos dataset será feita considerando que a chave primária do dataset Vivino é composta pelos campos **full_name** e **year**. Primeiramente são feitas transformações no dataset Winemag para que possuir campos equivalentes aos campos chave do Vivino. Para permitir isso, foram feitos os seguintes passos:
 - 1° passo: Extrair da coluna **Title** do dataset Winemag e informação de ano/safra, armazenar a informação em uma  coluna nova denomiada **year**.
 - 2° passo: Analisar registros individuais de ambos os datasets procurando encontrar padrões para identificar o campo **full_name** do Dataset Vivino a partir dos dados disponíveis no dataset Winemag (já que não há um campo de identificador comum entre os datasets).
 - 3° passo: Para cada nome candidato, criar a coluna com o nome e juntar os datasets. 
 - 4° passo: Unificar todos os datasets gerados no passo 3.

#### Winemag: criação da coluna **year** a partir da extraçào de informação contida em **Title**

In [4]:
# Analisando os samples do título, encontra-se o padrão regex (\s\d{4}\s) para a identificação de grupos de anos
with pd.option_context('display.max_colwidth', 150):
    print(winemag.Title.sample(20))

61170                             Jeff Cohn 2016 Rockpile Vineyard Haley Syrah (Rockpile)
6547                                        Carabella 2014 Plowbuster Pinot Noir (Oregon)
16436                                       Domaine des Marrans 2015  Beaujolais-Villages
16932                                                           Château Doms 2014  Graves
27212                               Torres 1995 Gran Sangre de Toro Reserva Red (Penedès)
12916                                                   Banfi 2013 Bel Nero Red (Toscana)
76679                            Domaine Guenaël Jambon 2016 Côte du Py Réserve  (Morgon)
54072                                                  Perimeter 2017 Merlot (Washington)
81464                             Château Labégorce Margaux 2017 Barrel Sample  (Margaux)
36950       B.R. Cohn 2010 Olive Hill Estate Vineyards Cabernet Sauvignon (Sonoma Valley)
83514                                              Brunelli 2015 Pariondo  (Valpolicella)
65024     

In [5]:
# Dos 87.259 registros, 381 que não possuem o padrão de ano no campo Title. Após a extração dos dados 
# esses vinhos ficarão com a nova coluna year nula e não serão levados na junção com o dataset Vivino.
condition = winemag.Title.str.contains(r'(\s\d{4}\s)', regex=True)
with pd.option_context('display.max_colwidth', 150):
    print(winemag.Title[ ~condition].sample(20))

24122      Charles B. Mitchell NV Bella Rossa Italian Style Red (Fair Play)
44926                   Hood Crest NV Grand Chêne Red (Columbia Gorge (OR))
55685                                      Frey NV Organic Red (California)
60375       Hound's Tree NV Highwire Pinot Noir (North Fork of Long Island)
39286                           Joullian NV Retro Rouge Red (Carmel Valley)
62754                                   Troublemaker NV Red (Central Coast)
20246                         Graveyard NV Paso Tombstone Red (Paso Robles)
35370                        Cairdeas NV MV Trí3 Red (Columbia Valley (WA))
17848                             Liberty Lake NV Legacy Red (Red Mountain)
68377                William Chris NV Skeleton Key Proprietor's Red (Texas)
17710          L'original French Kiss NV Cabernet Sauvignon (Vin de France)
78595                    Left Bend NV Mashup v.3 Red (Santa Cruz Mountains)
4401                         Original House Wine NV Merlot (Central Valley)
57263       

C:\Users\HAL9000\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [6]:
# Extração e criação da nova coluna year
winemag['year'] = winemag.Title.str.extract(r'(\s\d{4}\s)').replace(' ', '').astype('float64')

In [7]:
winemag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87216 entries, 0 to 87215
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        87216 non-null  object 
 1   Description  87216 non-null  object 
 2   Taster       84067 non-null  object 
 3   Rating       87216 non-null  int64  
 4   Price        87216 non-null  float64
 5   Designation  63064 non-null  object 
 6   Variety      87216 non-null  object 
 7   Appellation  87216 non-null  object 
 8   Winery       87216 non-null  object 
 9   Alcohol      82872 non-null  float64
 10  Bottle Size  87216 non-null  int64  
 11  year         86835 non-null  float64
dtypes: float64(3), int64(2), object(7)
memory usage: 8.0+ MB


In [8]:
winemag.loc[:, ['year', 'Title']].sample(10)

,year,Title
71920,2004.0,Handley 2004 Pinot Noir (Mendocino County)
55470,2017.0,Huett 2017 Shumaker Vineyard Pinot Noir (Willa...
18408,2015.0,Château Bélair-Monange 2015 Barrel Sample (Sa...
78720,2015.0,Sass 2015 Estate Pinot Noir (Willamette Valley)
14036,2015.0,Parras Wines 2015 Mula Velha Reserva Red (Lisboa)
31182,2013.0,Savage Grace 2013 Les Collines Vineyard Syrah ...
74468,1996.0,Brutocao 1996 Brutocao Vineyards Merlot (Mendo...
57787,2014.0,La Poderina 2014 Brunello di Montalcino
70490,2016.0,Montes 2016 Malbec (Colchagua Valley)
63411,2018.0,Château Clerc Milon 2018 Barrel Sample (Pauil...


#### Winemag: criação de colunas candidatas a junção com a coluna full_name(Vivino)

- Padrões identificados na coluna full_name para criação dos nomes candidatos, com exemplos.
| # | Title (Winemag) - example | full_name (Vivino) - example | name composition - example| candidate name pattern | matches |
| --- | --- | --- | --- | --- | --- |
| 1 | Scott Paul 2016 Maresh Vineyard Pinot Noir (Dundee Hills) | Scott Paul Maresh Vineyard Pinot Noir  | *Winery*: Scott Paul <br> *Designation*: Maresh Vineyard <br> *Variety*: Pinot Noir | [Winery] + [Designation] + [Variety] | 7.042 |
| 2 | Marco Abella 2013 Mas Mallola Red (Priorat) | Marco Abella Mas Mallola | *Winery*: Marco Abella <br> *Designation*: Mas Mallola | [Winery] + [Designation] | 2.861 |
| 3 | Saddleback 2011 Cabernet Sauvignon (Oakville) | Saddleback Cabernet Sauvignon | *Winery*: Saddleback <br> *Variery*: Cabernet Sauvignon | [Winery] + [Variety] | 8.892 |
| 4 | Apaltagua 2015 Reserva Carmenère (Apalta) | Apaltagua Carmenère Reserva  | *Winery*: Apatagua <br> *Designation*: Reserva <br> *Variety*: Carmenère | [Winery] + [Variety] + [Designation] | 272 |
| 5 | Château Feytit-Clinet 2017 Barrel Sample  (Pomerol) | Château Feytit-Clinet Pomerol | *Winery*: Château Feytit-Clinet <br> *Appellation*: Pomerol, Bordeaux, France | [Winery] + primeira parte de [Appellation] | 6.256 |
| 6 | Château du Caillau 2014 Cahors | Château du Caillau Malbec Cahors | *Winery*: Château du Caillau <br> *Variety*: Malbec <br> *Appellation*: Cahors, Southwest France, France | [Winery] + [Variety] + primeira parte de [Appellation] | 176 |
| 7 | Chime 2013 Cabernet Sauvignon (California) | Chime California Cabernet Sauvignon 	| *Winery*: Chime <br> *Appellation*: California, California Other, California, US <br> *Variety*: Cabernet Sauvignon | [Winery] + primeira parte de [Appellation] + [Variety] | 876 |
| 8 | Marcarini 2013 La Serra (Barolo) | Marcarini La Serra Barolo | *Winery*: Marcarini <br> *Designation*: La Serra <br> *Appellation*: Roero, Piedmont, Italy | [Winery] + [Designation] + primeira parte de [Appellation] 	 | 1.310 |
| 9 | Demarie 2011 Riserva (Roero) | Demarie Roero Riserva | *Winery*: Demarie <br> *Appellation*: California, California Other, California, US <br> *Designation*: Riserva | [Winery] + primeira parte de [Appellation] + [Designation] | 639 |
| 10 | Algodon 2016 Estate Bonarda (Mendoza) | Algodon Estate Bonarda | *Winery*: Algodon <br> *Designation*: Estate <br> *Variety*: Bonarda, Other Red | [Winery] + [Designation] + primeira parte de [Variety] | 360 |
| 11 | Lamadrid 2015 Single Vineyard Bonarda (Agrelo) | Lamadrid Bonarda Single Vineyard | *Winery*: Lamadrid <br> *Variety*: Bonarda, Other Red <br> *Designation*: Single Vineyard | [Winery] + primeira parte de [Variety] + [Designation] | 73 |
| 12 | Planeta 2013 Frappato (Vittoria) | Planeta Frappato | *Winery*: Planeta <br> *Variety*: Frappato, Italian Red | [Winery] + primeira parte de [Variety] | 549 |
| 13 | Scala Dei 2016 Garnatxa (Priorat) | Scala Dei Garnatxa Priorat | *Winery*: Scala Dei <br> *Variety*: Garnatxa, Grenache <br> *Appellation*: Priorat, Catalonia, Spain | [Winery] + primeira parte de [Variety] + primeira parte de [Appellation] | 35 |
| 14 | Kirkland Signature 2014 Meritage (Rutherford) | Kirkland Signature Rutherford Meritage | *Winery*: Kirkland Signature <br> *Appellation*: Rutherford, Napa, California, US <br> *Variety*: Meritage, Bordeaux-style Red Blend | [Winery] + primeira parte de [Appellation] + primeira parte de [Variety] | 55 |
| **Total** | --- | --- | --- | --- | **29.396** |

In [9]:
# Imprimindo exemplos de títulos para investigar os padrões de nomenclatura do Vivino
with pd.option_context('display.max_colwidth', 150):
    print(winemag.Title.sample(10))

64238                                       Mitolo 2014 Marsican Shiraz (McLaren Vale)
52232                      Hierogram 2016 Vineyard 8 Block N Old Vine Zinfandel (Lodi)
18135                                         Giacomo Fenocchio 2013 Nebbiolo (Langhe)
1508                                  Pierre Amadieu 2017 Les Piallats Red (Vinsobres)
31308                                            Joseph Drouhin 2012  Grands-Echezeaux
64765                                            Apothic 2015 Inferno Red (California)
70796                            Ceago Vinegarden 2000 Petite Sirah (Mendocino County)
67779                        Cantina del Taburno 2015 Fidelis  (Aglianico del Taburno)
82548                                Lange 2014 Reserve Pinot Noir (Willamette Valley)
55416    Cardwell Hill 2016 Monet Blocks Estate Bottled Pinot Noir (Willamette Valley)
Name: Title, dtype: object


In [10]:
winemag.loc[32091]
#winemag[ winemag['Winery'] == '']

Title          Paul Hobbs & Bertrand Gabriel Vigouroux 2011 C...
Description    One of two wines produced as a joint venture b...
Taster                                                Roger Voss
Rating                                                        91
Price                                                     62.625
Designation                                               Crocus
Variety                                                   Malbec
Appellation                     Cahors, Southwest France, France
Winery                   Paul Hobbs & Bertrand Gabriel Vigouroux
Alcohol                                                     14.5
Bottle Size                                                  750
year                                                      2011.0
Name: 32091, dtype: object

In [11]:
vivino[vivino['full_name'] == 'V. Sattui Napa Valley Syrah'].head(1)

,id,full_name,name,wine_region,winery,winery_region,country,ratings_count,ratings_average,alcohol,acidity,intensity,sweetness,tannin,year,vintage_id,vintage_ratings_count,vintage_ratings_average
815666,1761,V. Sattui Napa Valley Syrah,Napa Valley Syrah,Napa Valley,V. Sattui,St. Helena,United States,1103,3.9,14.8,3.144093,4.510927,1.267883,3.663588,2017,152160168,17,4.2


In [12]:
# Removendo espaços em branco do início e do fim de cada coluna usada
winemag['Winery'] = winemag['Winery'].str.strip()
winemag['Variety'] = winemag['Variety'].str.strip()
winemag['Designation'] = winemag['Designation'].str.strip()
winemag['Appellation'] = winemag['Appellation'].str.strip()
vivino['full_name'] = vivino['full_name'].str.strip()

# Criação de novas colunas com apenas a primeira parte da informação das colunas Appellation e Variety
winemag['first_appellation'] = winemag['Appellation'].str.extract(r'([\w ]+),', expand=False)
winemag['first_variety'] = winemag['Variety'].str.extract(r'([\w ]+),', expand=False)

# Definição da lista com os possíveis padrões para identificação do nome dos vinho no dataset da Vivino 
# a partir dos dados da Wine Enthusiast
candidate_patterns = [
    ['Winery', 'Designation', 'Variety'],
    ['Winery', 'Designation', ''],
    ['Winery', 'Variety', ''],
    ['Winery', 'Variety', 'Designation'],
    ['Winery', 'first_appellation', ''],
    ['Winery', 'Variety', 'first_appellation'],
    ['Winery', 'first_appellation', 'Variety'],
    ['Winery', 'Designation', 'first_appellation'],
    ['Winery', 'first_appellation', 'Designation'],
    ['Winery', 'Designation', 'first_variety'],
    ['Winery', 'first_variety', 'Designation'],
    ['Winery', 'first_variety', ''],
    ['Winery', 'first_variety', 'first_appellation'],
    ['Winery', 'first_appellation', 'first_variety']
]
red_wines_dfs = []
i = 0
for cp in candidate_patterns:
    if cp[2]:
        winemag['candidate'] = winemag.apply(\
                     lambda row: '{} {} {}'.format(row[cp[0]], row[cp[1]], row[cp[2]]) , axis = 1)
    else:
        winemag['candidate'] = winemag.apply(\
                     lambda row: '{} {}'.format(row[cp[0]], row[cp[1]]) , axis = 1)
    red_wines_dfs.append(\
        pd.merge(winemag, vivino,  how='inner', left_on=['candidate','year'], right_on = ['full_name','year']))
    # Atualiza dataset Winemag removendo os registro já unificados com Vivino
    winemag = winemag[~winemag['Title'].isin(list(red_wines_dfs[i].Title))]
    i += 1

i = 0
for df in red_wines_dfs:
    print('Pattern : {}\nEncontrados e combinados {} registros.\n'.format(candidate_patterns[i], red_wines_dfs[i].shape[0]))
    i += 1

Pattern : ['Winery', 'Designation', 'Variety']
Encontrados e combinados 7042 registros.

Pattern : ['Winery', 'Designation', '']
Encontrados e combinados 2861 registros.

Pattern : ['Winery', 'Variety', '']
Encontrados e combinados 8892 registros.

Pattern : ['Winery', 'Variety', 'Designation']
Encontrados e combinados 272 registros.

Pattern : ['Winery', 'first_appellation', '']
Encontrados e combinados 6256 registros.

Pattern : ['Winery', 'Variety', 'first_appellation']
Encontrados e combinados 176 registros.

Pattern : ['Winery', 'first_appellation', 'Variety']
Encontrados e combinados 876 registros.

Pattern : ['Winery', 'Designation', 'first_appellation']
Encontrados e combinados 1310 registros.

Pattern : ['Winery', 'first_appellation', 'Designation']
Encontrados e combinados 639 registros.

Pattern : ['Winery', 'Designation', 'first_variety']
Encontrados e combinados 360 registros.

Pattern : ['Winery', 'first_variety', 'Designation']
Encontrados e combinados 73 registros.

Pat

In [13]:
# Unificação dos datasets gerados a partir dos padrões identificados
red_wines = pd.concat(red_wines_dfs).drop(['candidate', 'first_appellation', 'first_variety'],axis=1)\
                .reset_index() 

In [14]:
print('Número de registros combinados: {}'.format(red_wines.shape[0]))
print('Percentual de registros combinados: {0:.2%}'.format((red_wines.shape[0]) / 87259))    

Número de registros combinados: 29396
Percentual de registros combinados: 33.69%


### Tratamento de dados omissos
- O campo **Alcohol** (Winemag) possui 2.31% dos registros (680) com valor nulo. O campo **alcohol** (Vivino), a uma primeira vista, não possui valores nulos, porém 8.437 registros possuem valor zero; e na verdade são oriundos de valores omissos.
 - O campo **alcohol** foi usado para preencher os valores omissos de **Alcohol**. Os valores preenchidos com dados inconsistentes (maiores que 23% ou menos que 5.5%) foram substituídos pela média do dataset. 
   - [5.5% - 23%] - Intervalo de variação para teor alcoólico de vinhos. https://winefolly.com/tips/the-lightest-to-the-strongest-wine/
- Os campos **Taster** e **Designation** possuem alguns valores nulos, mas não foram tratados pois não serão usados para a construção dos modelos de aprendizagem.

In [15]:
# Verificando registros duplicados
print('Duplicados em todas as colunas: ', red_wines.duplicated().sum())
print('Duplicados pelo título: ', red_wines.duplicated(subset=['Title']).sum())
print('Duplicados pelas colunas usadas para identificação: ', red_wines.duplicated(subset=['year', 'Winery', 'Appellation', 'Variety', 'Designation']).sum())

Duplicados em todas as colunas:  0
Duplicados pelo título:  0
Duplicados pelas colunas usadas para identificação:  0


In [16]:
# Verificando registros omissos
values_missing(red_wines, only_missing=False)

,percent_missing,rows_missing
Designation,31.136889,9153
Alcohol,2.313240,680
Taster,1.183835,348
index,0.000000,0
winery,0.000000,0
vintage_ratings_count,0.000000,0
vintage_id,0.000000,0
tannin,0.000000,0
sweetness,0.000000,0
intensity,0.000000,0


In [17]:
(red_wines['alcohol'] == 0).sum()

8436

In [18]:
# Preenche valores omissos de Alcohol com os dados de alcohol 
# e substituí valores inconsistentes (menores que 5.5% ou maiores que 23%) pela média
alcohol = red_wines.apply(lambda x: x['alcohol'] if pd.isnull(x['Alcohol']) else x['Alcohol'], axis=1).\
    apply(lambda x: x if (x >= 5.5) and (x <= 23) else np.nan)
red_wines['Alcohol'] = alcohol.fillna(alcohol.mean())

In [19]:
red_wines['Alcohol'].describe()

count    29396.000000
mean        14.087464
std          0.685622
min          8.500000
25%         13.500000
50%         14.087464
75%         14.500000
max         17.400000
Name: Alcohol, dtype: float64

### Correções e transformações nos dados
 - Criação da coluna **continent** aplicando a função *country_to_continent*. Sintetizei a informação do páis, criando a coluna com informação do continente, com o intuito de ter dados mais generalizados para uso nos modelos. Percebeu-se ao analisar a média por país e a média continental da avaliação do vinho que os valores por país em cada continente são relativamente semelhantes.

In [20]:
print(red_wines.country.value_counts())
print('\n', red_wines.groupby('country')['Rating'].mean())

United States    14207
Italy             6686
France            2610
Spain             1153
Argentina         1089
Australia          893
Chile              817
Portugal           637
New Zealand        418
South Africa       295
Austria            246
Israel             100
Germany             79
Uruguay             70
Canada              65
Brazil              16
Greece              10
Switzerland          4
Hungary              1
Name: country, dtype: int64

 country
Argentina        87.960514
Australia        89.866741
Austria          91.191057
Brazil           83.687500
Canada           89.215385
Chile            87.348837
France           89.175479
Germany          91.696203
Greece           88.400000
Hungary          89.000000
Israel           90.050000
Italy            90.016452
New Zealand      89.241627
Portugal         90.100471
South Africa     89.979661
Spain            88.583695
Switzerland      89.500000
United States    90.073837
Uruguay          87.857143
Name: Rating

In [21]:
def country_to_continent(country):
    '''Retorna o continente a partir do nome do país'''
    south_america = ['Argentina', 'Brazil', 'Chile', 'Uruguay']
    north_america = ['Canada', 'United States']
    oceania = ['Australia', 'New Zealand']
    africa = ['South Africa']
    asia = ['Israel']
    if country in south_america:
        return 'South America'
    elif country in north_america:
        return 'North America'
    elif country in oceania:
        return 'Oceania'
    elif country in africa:
        return 'Africa'
    elif country in asia:
        return 'Asia'
    else:
        return 'Europe'

In [22]:
red_wines['continent'] = red_wines['country'].apply(country_to_continent)

In [23]:
red_wines.groupby('continent')['Rating'].mean()

continent
Africa           89.979661
Asia             90.050000
Europe           89.719674
North America    90.069927
Oceania          89.667429
South America    87.671687
Name: Rating, dtype: float64

- Criacão das colunas **popular_wine** e **popular_vintage** a partir dos campos **ratings_count** e **vinage_ratings_count**, a fim de representar se determinado vinho e safra, respectivamente, são populares (popular definido como sendo número de avaliações maior do que a média).

In [24]:
red_wines['popular_wine'] = red_wines['ratings_count'].apply(\
                                lambda x: 1 if (x >= red_wines['ratings_count'].mean()) else 0)
red_wines['popular_vintage'] = red_wines['vintage_ratings_count'].apply(\
                                lambda x: 1 if (x >= red_wines['vintage_ratings_count'].mean()) else 0)

- Criação da coluna **rating_group** com base no valor de Rating agrupado de acordo com o guia de interpretação da nota publicado no site da Wine Enthusiast. <br><br>
 
- Grupo **0** --> 98–100: Classic "The pinnacle of quality"
- Grupo **1** --> 94–97: Superb "A great achievement" 
- Grupo **2** --> 90–93: Excellent "Highly recommended"
- Grupo **3** --> 87–89: Very Good "Often good value; well recommended"
- Grupo **4** --> 83–86: Good "Suitable for everyday consumption; often good value"
- Grupo **5** --> 80–82: Acceptable "Can be employed in casual, less-critical circumstances"
<br>
https://www.winemag.com/our-buying-guide-and-blind-tasting-process/

In [25]:
# Conversão de ratings para rating group

def ratings_to_group(rating):
    if rating in range(80,83):
        return 0
    elif rating in range(83,87):
        return 1
    elif rating in range(87,90):
        return 2
    elif rating in range(90,94):
        return 3
    elif rating in range(94,98):
        return 4
    else:
        return 5
    
red_wines['rating_group'] = red_wines['Rating'].apply(ratings_to_group)

### Remoção de colunas
- Os seguintes campos foram removidas do dataset pelos motivos abaixo:
 - Campos de identificação do vinho/safra: **index**, **id**, **vintage_id** e **Title**.
 - Campos com informações textuais do vinho e do review que não serão transformadas em categorias: **Taster**, **name**, **wine_region**, **winery** e **winery_region**.
 - Campos usados para unificação dos datasets: **Designation**, **Variety**, **Appellation**, **Winery**, **full_name** e **year**.
 - Campos usados em transformações de outros valores: **Bottle Size**, **alcohol**, **ratings_count**, **vintage_ratings_count**, **country** e **Rating**.

In [26]:
red_wines.drop(
    ['index', 'id', 'vintage_id', 'Title',
     'Taster', 'name','wine_region', 'winery', 'winery_region',
     'Designation', 'Variety', 'Appellation', 'Winery', 'full_name', 'year',
     'Bottle Size', 'alcohol', 'country', 'ratings_count','vintage_ratings_count', 'Rating'], 
    axis = 1, inplace=True
)

### Exportação do dataset

In [27]:
# Exporta o dataset para arquivo csv após o tratamento dado
#red_wines.to_csv('datasets/red_wines.csv', index=False, header=True)

In [28]:
red_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29396 entries, 0 to 29395
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Description              29396 non-null  object 
 1   Price                    29396 non-null  float64
 2   Alcohol                  29396 non-null  float64
 3   ratings_average          29396 non-null  float64
 4   acidity                  29396 non-null  float64
 5   intensity                29396 non-null  float64
 6   sweetness                29396 non-null  float64
 7   tannin                   29396 non-null  float64
 8   vintage_ratings_average  29396 non-null  float64
 9   continent                29396 non-null  object 
 10  popular_wine             29396 non-null  int64  
 11  popular_vintage          29396 non-null  int64  
 12  rating_group             29396 non-null  int64  
dtypes: float64(8), int64(3), object(2)
memory usage: 2.9+ MB
